In [34]:
import pyspark.sql.functions as f

from pyspark.sql import SparkSession, SQLContext


spark = SparkSession \
    .builder \
    .appName("PySpark") \
    .getOrCreate()

sc = spark.sparkContext
sql_c = SQLContext(spark.sparkContext)


In [ ]:
#Téléchargement des fichiers puis copie vers HDFS

In [31]:
import os
import urllib.request
from subprocess import run

# URL du fichier à télécharger
url = "https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/projects/9c15cb/sample.csv"

# Nom du fichier de destination
destination_file = os.path.join( "sample.csv")

try:
    # Télécharger le fichier seulement s'il n'existe pas déjà
    if not os.path.exists(destination_file):
        urllib.request.urlretrieve(url, destination_file)
        print(f"Le fichier {destination_file} a été téléchargé avec succès.")
    else:
        print(f"Le fichier {destination_file} existe déjà.")

    # Déposer le fichier sur HDFS
    hdfs_destination = "/sample.csv"  # Spécifiez le chemin sur HDFS
    run(["hadoop", "fs", "-copyFromLocal", destination_file, hdfs_destination])
    print(f"Le fichier a été déposé avec succès sur HDFS à {hdfs_destination}")
except Exception as e:
    print(f"Une erreur s'est produite : {str(e)}")


Le fichier sample.csv existe déjà.
Le fichier a été déposé avec succès sur HDFS à /sample.csv


In [36]:
# Charger le fichier CSV depuis le système de fichiers local
csv_path = os.path.expanduser("/sample.csv")
data = spark.read.csv(csv_path, header=True)

# Afficher les 10 premières lignes
lookup.show(10)


+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:00|      view|  44600062|2103807459595387724|                null|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:00|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|   33.2|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:01|      view|  17200506|2053013559792632471|furniture.living_...|    null|  543.1|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:01|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:04|      view|   1004237|2053013555631882655|electr

In [52]:
#Conversion des colonnes
data = data.withColumn("event_time", f.col("event_time").cast("timestamp"))\
           .withColumn("product_id", f.col("product_id").cast("int"))\
           .withColumn("category_id", f.col("product_id").cast("int"))\
           .withColumn("price", f.col("price").cast("float"))\
           .withColumn("user_id", f.col("user_id").cast("int"))
data.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: float (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [54]:
data.count()

1244245